In [3]:
import praw

In [4]:
reddit = praw.Reddit(client_id = 'Tc_MOdA4g5Kc1x5_krKujQ',
                        client_secret = 'Vpqq2vxJpqcVk_pUJjiCJ9-qbfXAqw',
                        user_agent = 'jingtingxu', check_for_async=False)

subreddit = "ChatGPT"
breadthCommentCount = 10
targetSub = reddit.subreddit(subreddit)

In [ ]:
pip install torch-geometric praw

In [ ]:
import torch
from torch_geometric.data import Data
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize lists for node and edge data
post_author = []
post_author_karma = []
post_upvotes = []
post_upvotes_bin = []
post_upvote_ratio = []
post_upvote_ratio_bin = []
post_titles = []
post_contents = []
edge_index = []
comment_author = []
comment_author_karma = []
comment_upvotes = []
comment_upvotes_bin = []

# Loop over posts and comments
node_labels = []
post_count = 0

# Define a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Define a threshold for the cosine similarity
similarity_threshold = 0.3

for post in tqdm(targetSub.hot(limit=200)):
    if post.selftext and post.author and post.comments:
        post_author.append(post.author.name)
        post_author_karma.append(post.author.comment_karma)
        post_upvotes.append(post.score)
        post_upvote_bin = 1 if post.score > 3 else 0
        post_upvotes_bin.append(post_upvote_bin)
        post_upvote_ratio.append(post.upvote_ratio)
        upvote_bin = 1 if post.upvote_ratio > 0.5 else 0
        post_upvote_ratio_bin.append(upvote_bin)
        
        # Add the post title and content to the list
        post_titles.append(post.title)
        post_contents.append(post.selftext)

        comment_count = 0
        post.comments.replace_more(limit=None)
        for comment in post.comments.list():
            if isinstance(comment, praw.models.MoreComments):
                continue
            if comment.author:
                comment_author.append(comment.author.name)
                comment_author_karma.append(comment.author.comment_karma)
                comment_upvotes.append(comment.score)
                com_upvote_bin = 1 if comment.score > 3 else 0
                comment_upvotes_bin.append(com_upvote_bin)

                edge_index.append([post_count + comment_count, post_count])
                
                # Add comment label to list
                node_labels.append(com_upvote_bin)
                
                comment_count += 1
#                 if comment_count == 10:
#                     break

        # Add post label to list
        node_labels.append(post_upvote_bin)

        post_count += 1

# Create node features tensor for posts
post_features = torch.tensor(list(zip(post_author_karma, post_upvote_ratio, post_upvotes_bin)), dtype=torch.float)

# Create node features tensor for comments
comment_features = torch.tensor(list(zip(comment_author_karma, comment_upvotes, comment_upvotes_bin)), dtype=torch.float)

# Calculate the TF-IDF matrix for post titles and contents
tfidf_matrix = vectorizer.fit_transform(post_titles + post_contents)

# Calculate the cosine similarity matrix for the posts
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

# Loop over the cosine similarity matrix and add edges for similar posts
for i in range(len(post_titles)):
    for j in range(i+1, len(post_titles)):
        if cosine_sim_matrix[i][j] > similarity_threshold:
            edge_index.append([i, j])

# Create edge index tensor
edge_index = torch.tensor(edge_index).t()

# Create node type tensor
post_node_types = torch.zeros((post_count, 1), dtype=torch.long)
comment_node_types = torch.ones((len(comment_author), 1), dtype=torch.long)
node_types = torch.cat([post_node_types, comment_node_types])

# Concatenate post and comment feature and label tensors
node_features = torch.cat([post_features, comment_features])
node_labels_tensor = torch.tensor(node_labels, dtype=torch.long)

5it [17:31, 164.78s/it]

In [323]:
# Create PyTorch Geometric Data object
data = Data(x=torch.cat((post_features, comment_features), dim=0), edge_index=edge_index, y=node_labels, node_type=node_types)

In [327]:
# Save data object to file
torch.save(data, 'my_data.pt')

In [329]:
# Split DataLoader object into 60% training, 20% validation, and 20% test sets
num_data = data.num_nodes
num_train = int(num_data * 0.6)
num_val = int(num_data * 0.2)
num_test = num_data - num_train - num_val

# train_data, val_data, test_data = random_split(range(loader.dataset.num_nodes), [num_train, num_val, num_test])

# # Convert node labels to one-hot vectors
# num_classes = 2

# train_labels = torch.nn.functional.one_hot(torch.tensor(train_data.dataset.y), num_classes=num_classes)
# val_labels = torch.nn.functional.one_hot(torch.tensor(val_data.dataset.y), num_classes=num_classes)

In [330]:
train_idx, val_data, test_idx = random_split(range(data.num_nodes), [num_train, num_val, num_test])

## Graph Neural Network for Node Classification

In [265]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv #GATConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # Initialize the layers
        self.conv1 = GCNConv(data.num_features, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, hidden_channels) # I am using a single message passing layer 
# since I have disconnected graphs
        self.out = Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # First Message Passing Layer (Transformation)
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

#         # Second Message Passing Layer
#         x = self.conv2(x, edge_index)
#         x = x.relu()
#         x = F.dropout(x, p=0.5, training=self.training)

        # Output layer 
        x = F.softmax(self.out(x), dim=1)
        return x

model = GCN(hidden_channels=4)
print(model)

GCN(
  (conv1): GCNConv(3, 4)
  (out): Linear(in_features=4, out_features=2, bias=True)
)


In [266]:
# data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[train_idx.indices], torch.tensor([data.y[i] for i in train_idx.indices]))
    print(loss.item())
    loss.backward()
    optimizer.step()

-0.5951312184333801
-0.5952305197715759
-0.609758198261261
-0.5724429488182068
-0.5928376913070679
-0.5794914364814758
-0.6067376136779785
-0.557999849319458
-0.574445366859436
-0.5670799016952515
-0.6439798474311829
-0.5350962281227112
-0.5448779463768005
-0.5309566259384155
-0.5449820160865784
-0.5769739747047424
-0.5333811044692993
-0.6339825987815857
-0.5252252221107483
-0.5664832592010498
-0.5609647035598755
-0.5534942746162415
-0.5322118997573853
-0.5554783940315247
-0.5437180995941162
-0.5805246829986572
-0.5328212976455688
-0.5421494245529175
-0.638243556022644
-0.6369972825050354
-0.6108854413032532
-0.5622504949569702
-0.5524722933769226
-0.5812246799468994
-0.5922462344169617
-0.5061576962471008
-0.5037481188774109
-0.5624899864196777
-0.5320104360580444
-0.5546674132347107
-0.5622579455375671
-0.5865097641944885
-0.5411204099655151
-0.5769769549369812
-0.5251338481903076
-0.5267313122749329
-0.5521610975265503
-0.5640000700950623
-0.5374433994293213
-0.564220666885376
-0.51

In [253]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

AttributeError: 'GlobalStorage' object has no attribute 'test_mask'